In [ ]:
from dotenv import load_dotenv
from polygon import RESTClient
from datetime import datetime,timezone
import os

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:

load_dotenv("../guidelight-api/.env")
token = os.getenv("POLYGON_TOKEN")

In [ ]:

client = RESTClient(api_key=token)

In [ ]:
ticker = "AAPL"

# List Aggregates (Bars)
aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2019-01-01", to="2023-12-30", limit=50000):
    aggs.append(a)

print(len(aggs))



In [ ]:
aggs[-1]

In [ ]:
#unix millisecond timestamp to datetime
# from pytz import timezone

# Convert timestamp to datetime in UTC timezone
def ts_to_dt(timestamp):
    return datetime.fromtimestamp(timestamp/1000, tz=timezone.utc)

timestamps = np.vectorize(ts_to_dt)

# Example usage
a = np.array([a.timestamp for a in aggs])
times = timestamps(a)
aapl_aggs = pd.DataFrame({"day": [time.strftime("%Y-%m-%d") for time in times],
                            "timestamp": [time.strftime("%Y-%m-%d, %H:%M") for time in times],
                           "close": [a.close for a in aggs], 
                           "volume": [a.volume for a in aggs],
                           "low": [a.low for a in aggs],
                           "high": [a.high for a in aggs]})



In [ ]:
from heston_param import *

In [ ]:
aapl_aggs.groupby("day")

In [ ]:
daily_aggs =  [aapl_aggs.groupby('day').get_group(x) for x in aapl_aggs['day'].unique()]

In [ ]:
hist_voilatilities = torch.empty(len(daily_aggs))
for i, day in enumerate(daily_aggs):
	hist_voilatilities[i] = estimate_historical_volatility(day['close'].values)

# calculate the historical volatility for each day





In [ ]:
import pickle

pickle.dump(hist_voilatilities, open("hist_voilatilities.pkl", "wb"))
pickle.dump(daily_aggs, open("daily_aggs.pkl", "wb"))

In [45]:
hist_voilatilities = pickle.load(open("hist_voilatilities.pkl", "rb"))
daily_aggs = pickle.load(open("daily_aggs.pkl", "rb"))

In [49]:
print(calibrate_daily_parameters(hist_voilatilities[0], 0.1, daily_aggs[0]["close"].values, 0.0237, 390, 3))

/Users/brad/mlprojects/guidelight/inference/heston_param.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  median_simulated_vols[i] = torch.median(torch.median(torch.tensor(simulated_vols), dim=0).values)  # Ensure conversion to tensor if needed
/Users/brad/mlprojects/guidelight/inference/heston_param.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.mean((median_simulated_vols - torch.tensor(historical_vol)) ** 2)  # Ensure historical_vol is correctly handled


KeyboardInterrupt: 